In [1]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage

In [2]:
df = pd.read_csv('featurized_condensed.csv')

In [3]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 10', 'Ingredients'])

In [4]:
df = df.set_index('ID')

In [5]:
cuisines = list(set(df['Cuisine']))

In [6]:
binary_cuisines = {}
counter = 0
for i in cuisines:
    length = np.zeros(len(cuisines))
    length[counter] = 1
    length = length.tolist()
    counter += 1
    binary_cuisines[i] = length

In [7]:
lists = {}
for i in df.iterrows():
    i[1][0] = binary_cuisines[i[1][0]]
    lists[i[0]] = i[1][0]+i[1][1:].get_values().tolist()

In [8]:
ncuisines = len(df['Cuisine'].unique())
print(ncuisines)
print(df['Cuisine'].unique())
nattributes = len(df.columns[1:7])
print(nattributes)
print(np.array(df.columns[1:7]))
ningredients = len(df.columns[7:])
print(ningredients)
print(df.columns[7:])

20
['thai' 'korean' 'japanese' 'cajun_creole' 'italian' 'indian' 'vietnamese'
 'jamaican' 'british' 'mexican' 'southern_us' 'chinese' 'greek' 'filipino'
 'moroccan' 'brazilian' 'irish' 'french' 'spanish' 'russian']
6
['Vegeterian (Y/N)' 'Vegan (Y/N)' 'Dessert (Y/N)' 'Spicy (Y/N)'
 'Drink (Y/N)' 'Alcohol (Y/N)']
395
Index(['paprika', 'quail eggs', 'sweet chili sauce', 'half & half',
       'allspice berries', 'carrots', 'peppercorns', 'cardamom',
       'vanilla ice cream', 'red chili',
       ...
       'paneer', 'cheese', 'onion powder', 'cauliflower', 'lemon juice',
       'vinegar', 'tumeric', 'back bacon rashers', 'plum tomatoes', 'beets'],
      dtype='object', length=395)


In [9]:
binary_df = pd.DataFrame(lists)
input_pats = binary_df.iloc[:,:ncuisines+nattributes]
input_pats = torch.tensor(np.array(input_pats),dtype=torch.float)
output_pats = binary_df.iloc[:,ncuisines+nattributes:]
output_pats = torch.tensor(np.array(output_pats),dtype=torch.float)
N = input_pats.shape[0] # number of training patterns
input_v = input_pats[0,:].numpy().astype('bool')
output_v = output_pats[0,:].numpy().astype('bool')

In [10]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        # Input
        #  rep_size : number of hidden units in "Representation Layer"
        #  hidden_Size : number of hidden units in "Hidden Layer"
        #
        # TODO : YOUR CODE GOES HERE    
        
        self.i2r = nn.Linear(ncuisines+nattributes, rep_size)
        self.r2h = nn.Linear(rep_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, ningredients)
        #raise Exception('Replace with your code.')

    def forward(self, x):
        # Defines forward pass for the network on input patterns x
        #
        # Input can take these two forms:
        #
        #   x: [nobj+nrel 1D Tensor], which is a single input pattern as a 1D tensor
        #      (containing both object and relation 1-hot identifier) (batch size is B=1)
        #   OR
        #   x : [B x (nobj+nrel) Tensor], which is a batch of B input patterns (one for each row)
        #
        # Output
        #   output [B x nattribute Tensor], which is the output pattern for each input pattern B on the Attribute Layer
        #   hidden [B x hidden_size Tensor], which are activations in the Hidden Layer
        #   rep [B x rep_size Tensor], which are the activations in the Representation LAyer
        x = x.view(-1,ncuisines+nattributes) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        x_item = x[:,:ncuisines+nattributes] # input to Item Layer [B x nobj Tensor]
#         print("x item shape "+ str(x_item.shape))
        #x_rel = x[:,ncuisines:] # input to Relation Layer [B x nrel Tensor]
        # TODO : YOUR CODE GOES HERE
        
        rep = self.i2r(x_item)
        rep = relu(rep)
        #combined = torch.cat((rep, x_rel),1)
        hidden = self.r2h(rep)
        hidden = relu(hidden)
        output = self.h2o(hidden)
        output = sigmoid(output)
        #raise Exception('Replace with your code.')

        return output, hidden, rep

In [11]:
def train(mynet,epoch_count,nepochs_additional=5000):
    # Input
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden, rep = mynet(input_pats[p,:]) # forward pass
            target = output_pats[p,:] 
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 50 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [12]:
def get_rep(net):
    # Extract the hidden activations on the Representation Layer for each item
    # 
    # Input
    #  net : Net class object
    #
    # Output
    #  rep : [nitem x rep_size numpy array], where each row is an item
    input_clean = torch.zeros(nobj,nobj+nrel)
    for idx,name in enumerate(names_items):
        input_clean[idx,idx] = 1. # 1-hot encoding of each object (while Relation Layer doesn't matter)
    output, hidden, rep = mynet(input_clean)
    return rep.detach().numpy()

def plot_rep(rep1,rep2,rep3,names):
    #  Compares Representation Layer activations of Items at three different times points in learning (rep1, rep2, rep3)
    #  using bar graphs
    # 
    #  Each rep1, rep2, rep3 is a [nitem x rep_size numpy array]
    #  names : [nitem list] of item names
    #
    nepochs_list = [nepochs_phase1,nepochs_phase2,nepochs_phase3]
    nrows = nobj
    R = np.dstack((rep1,rep2,rep3))    
    mx = R.max()
    mn = R.min()
    depth = R.shape[2]
    count = 1
    plt.figure(1,figsize=(4.2,8.4))
    for i in range(nrows):
        for d in range(R.shape[2]):
            plt.subplot(nrows, depth, count)
            rep = R[i,:,d]
            plt.bar(range(rep.size),rep)
            plt.ylim([mn,mx])
            plt.xticks([])
            plt.yticks([])        
            if d==0:
                plt.ylabel(names[i])
            if i==0:
                plt.title("epoch " + str(nepochs_list[d]))
            count += 1
    plt.show()

def plot_dendo(rep1,rep2,rep3,names):
    #  Compares Representation Layer activations of Items at three different times points in learning (rep1, rep2, rep3)
    #  using hierarchical clustering
    # 
    #  Each rep1, rep2, rep3 is a [nitem x rep_size numpy array]
    #  names : [nitem list] of item names
    #
    nepochs_list = [nepochs_phase1,nepochs_phase2,nepochs_phase3]
    linked1 = linkage(rep1,'single')
    linked2 = linkage(rep2,'single')
    linked3 = linkage(rep3,'single')
    mx = np.dstack((linked1[:,2],linked2[:,2],linked3[:,2])).max()+0.1    
    plt.figure(2,figsize=(7,12))
    plt.subplot(3,1,1)    
    dendrogram(linked1, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.title('Hierarchical clustering; ' + "epoch " + str(nepochs_list[0]))
    plt.ylabel('Euclidean distance')
    plt.subplot(3,1,2)
    plt.title("epoch " + str(nepochs_list[1]))
    dendrogram(linked2, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.subplot(3,1,3)
    plt.title("epoch " + str(nepochs_list[2]))
    dendrogram(linked3, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.show()

In [13]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

nepochs_phase1 = 500
nepochs_phase2 = 1000
nepochs_phase3 = 2500
epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase1)
rep1 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase2-nepochs_phase1)
rep2 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase3-nepochs_phase2)
rep3 = get_rep(mynet)


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: input and target shapes do not match: input [1 x 395], target [73] at /Users/soumith/miniconda2/conda-bld/pytorch_1532623076075/work/aten/src/THNN/generic/MSECriterion.c:12

In [ ]:
plot_rep(rep1,rep2,rep3,names_items)
plot_dendo(rep1,rep2,rep3,names_items)